# Machine learning using EKF

In [1]:
from IPython import display
import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from load import load_abalone_data,load_bikes_data
# Importing Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from  sklearn.datasets import make_regression
from sklearn.datasets import load_boston
from tqdm import tqdm as tqdm

In [2]:
# Select device which you are going to use for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


### Import Data Sets
Testing using a toy sine data

In [3]:
X,y = load_bikes_data()
print(y.shape)
print(X.shape)

(17379,)
(17379, 14)


## Data Partition


In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=.5)
X_val,X_test, y_val,y_test = train_test_split(X_test,y_test,test_size=0.5)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
# x_train_scaled = scaler.fit_transform(x_train)
# x_test_scaled = scaler.transform(x_test)

(8689, 14)
(4345, 14)
(4345, 14)


## Define Neural network

In [6]:
class MLP(nn.Module):
    def __init__(self, n_inputs, n_hidden_layer, n_outputs,bias=True):
        super(MLP, self).__init__()
        # YOUR CODE HERE
        #raise NotImplementedError()
        self.fc1 = nn.Linear(n_inputs, n_hidden_layer, bias)
        self.fc2 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc3 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc4 = nn.Linear(n_hidden_layer, n_outputs, bias)
        

    def forward(self, x):
        # YOUR CODE HERE
        #raise NotImplementedError()
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x    

In [7]:
## Testing net
n_inputs = X.shape[1]
n_outputs = 1
n_hidden_layer = 100
test_net = MLP(n_inputs, n_hidden_layer, n_outputs)
print(test_net)

MLP(
  (fc1): Linear(in_features=14, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
)


In [8]:
# Define number of Input and Output layers
torch.set_default_dtype(torch.float64)
n_inputs = X.shape[1]
n_outputs = 1
n_hidden_layer = 20
mlp = MLP(n_inputs,n_hidden_layer, n_outputs)
mpl = mlp.to(device)

optimizer = torch.optim.Adam(mlp.parameters(), lr=0.005)
n_epochs = 200

train_accuracy_history = []
test_accuracy_history = []
print_every = 500
x_tensor = torch.tensor(X_train, device=device, dtype=torch.float64)
y_tensor = torch.tensor(y_train, device=device, dtype=torch.float64)
x_val_tensor = torch.tensor(X_val, device=device, dtype=torch.float64)
print_every = 10
ceLoss = nn.MSELoss()



for epoch in (range(n_epochs)):
    
    #Set gradients as zero
    mlp.zero_grad()
    #Calculate outputs with forward()
    outputs = mlp(x_tensor)   
    
    #calculate loss
    loss = ceLoss(outputs, y_tensor)
#     print(f"The output is :{outputs.item()} expected is :{y_train}")
    #calculate gradients - calling backward()
    loss.backward()
    
    #update weights using optimizer
    optimizer.step()
    if(epoch%print_every ==0):
        with torch.no_grad():
            y_pred = mlp.forward(x_val_tensor)
            y_pred = y_pred.cpu().data.numpy()
            error = mean_squared_error(y_val,y_pred)
            val_error =  np.sqrt(error)
        print(f"sample:{epoch} train error: {loss.item()} val error:{val_error.item()}")

C:\Users\Ananth Mahadevan\.conda\envs\torch\lib\site-packages\torch\nn\modules\loss.py:443: UserWarning: Using a target size (torch.Size([8689])) that is different to the input size (torch.Size([8689, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


NameError: name 'mean_squared_error' is not defined

In [70]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
with torch.no_grad():
    x_test = torch.tensor(X_test, device=device, dtype=torch.float64)
    y_pred = mlp.forward(x_test)
    y_pred = y_pred.cpu().data.numpy()
    error = mean_squared_error(y_test,y_pred)
    print(np.sqrt(error))

173.80891063683873
